In [4]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern  

# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100, #100
                'lr':4e-4,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.1,
                #'set_spatial_units':  station,   
                #  
                'scheduler':None,
                #'scheduler':True,
                #'torch_scheduler_milestone': 5,
                #'torch_scheduler_gamma':0.997,
                #'torch_scheduler_lr_start_factor':1,

                'vision_num_heads':6,
                "vision_grn_out_dim":48,
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 



Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  FLA
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/

Loading BokehJS ...

Training and Hyper-parameter tuning with Ray is not possible

>>>> Load best CONFIG


Namespace(model_name='STGCN', dataset_names=['subway_in', 'netmob_POIs'], dataset_for_coverage=['subway_in', 'netmob_POIs'], n_vertex=40, C=1, device=device(type='cuda'), optimizer='adamw', single_station=False, loss_function_type='MSE', epsilon_clustering=0.05, contextual_positions={'netmob_POIs': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]}, quick_vision=False, netmob_transfer_mode='DL', evaluate_complete_ds=True, train_valid_test_split_method='similar_length_method', set_spatial_units=None, num_workers=0, persistent_workers=False, pin_memory=True, prefetch_factor=2, drop_last=False, mixed_precision=False, non_blocking=True, torch_compile=False, backend='inductor', prefetch_all=False, ray=False, ray_scheduler='ASHA', ray_search_alg=None, grace_period=20, HP_max_epochs=100, alpha=None, conformity_scores_type='max_residual', quantile_method='compute_quantile_by_class', calibrati

In [ ]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh


# Load config
model_name = 'STGCN' #'CNN'
dataset_names = ['subway_in','netmob']
args = get_args(model_name,dataset_names)

# Modification : 
args.K_fold = 5

args.ray = False
args.W = 0  # IMPORTANT AVEC NETMOB

args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

# optimization:
args.mixed_precision = True

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(args,folds)
K_subway_ds,args = K_fold_splitter.split_k_fold()
subway_ds = K_subway_ds[0]

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [10]:
import torch

netmob_init = torch.Tensor([7392*4*263*287])
netmob_tensor = torch.Tensor([2934*4*263*287*8])

print('Init tensor: ',netmob_init)
print('Init feature vect: ',netmob_tensor)

print('\nblock1:')
sconv = torch.prod(torch.Tensor([64, 32, 131, 143, 8]))
tconv = torch.prod(torch.Tensor([64, 128, 131, 143, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)

print('\nblock2:')
sconv = torch.prod(torch.Tensor([64, 658, 65, 71, 8]))
tconv = torch.prod(torch.Tensor([64, 512, 65, 71, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)


Init tensor:  tensor([2.2318e+09])
Init feature vect:  tensor([7.0868e+09])

block1:
s_conv:  tensor(3.0692e+08) tconv:  tensor(1.2277e+09)

block2:
s_conv:  tensor(1.5548e+09) tconv:  tensor(1.2098e+09)
